In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import os
import time
import pandas as pd
import numpy as np
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.common.action_chains import ActionChains

In [2]:
os.environ['PATH']+= r'C:/SeleniumDrivers'

In [3]:
driver=webdriver.Chrome()
driver.get('https://world.openfoodfacts.org/country/india')

In [4]:
elements=driver.find_elements(By.CSS_SELECTOR,'div.list_product_name')

In [5]:
for i in elements:
    print(i.text)

Thums Up (Coca-Cola)
Red bull energy drink - 250 ml
Sprite - coca cola - 750 ml
Heinz Tomato ketchup - 400ml
Thumbs Up - Coca Cola Company - 750ml
Sting - 250 ml
kinley mineral water - 1 l
Sprite - 2.25l
Mountain Dew - Pepsico
maaza - Hisdustan Unilever Limited - 600 ml
Kinley Mineral Water - 1 L
Limca - Coca-Cola
HERSHEY’S CHOCOLATE SYRUP - 623 g
Cadbury gems - 7.9g
Taka tak - Haldiram's
Thums up - 250 ml
Chyawanprash - Dabur - 550 g
Good Day Butter Cookies - Britannia - 68g
Sugar cookie - CMB - 100 g
Frooti - 2 l
Diet coke - The Coca-Cola Company - 500 ml
Mountain dew - mountaindew - 2.25
7up - PepsiCo - 750 ml
Coca Cola - 750 ml
Rooh Afza - hamdard - 750 ml
Marie Gold - Britannia - 73 g
Pepsi - 1,5 L
Pepsi 2.25 l - Pepsico
Parle - 22g
Bourn vita - 500g
Eno, fruit salt lemon flavour - GSK - 5g
Maaza Original - - 1.2 l
coco cola company - sprite - 250 ml
Barritas de platano y chocolate - Alteza
Oreo
Frooti mango drink - parle Agro - 150ml
thumsup - recycle
ORSL Plus Electrolyte Drink 

In [6]:
product_link = driver.find_elements(By.CSS_SELECTOR, 'a.list_product_a')

In [7]:
product_linkies=[]
for i in product_link:
    product_linkies.append(i.get_attribute('href'))

In [8]:
def individual():
    try:
        prod_name = driver.find_element(By.CSS_SELECTOR, 'h2.title-1').text.strip()
        rows = driver.find_elements(By.XPATH, "//tbody/tr")
        if not prod_name or not rows:
            return pd.DataFrame()
        nutrition_data = {"Product Name": [prod_name]}
        for row in rows:
            nutrient = row.find_element(By.XPATH, "./td[1]").text.strip()
            as_sold_100g = row.find_element(By.XPATH, "./td[2]").text.strip()
            nutrition_data[nutrient] = [as_sold_100g]
        df = pd.DataFrame(nutrition_data)
        return df
    except NoSuchElementException:
        return pd.DataFrame()

In [12]:
import numpy as np

master_df = pd.DataFrame()
for product_link in product_linkies:
    main_window = driver.current_window_handle
    driver.execute_script("window.open();")
    driver.switch_to.window(driver.window_handles[-1])
    driver.get(product_link)
    
    try:
        # Wait until the page is loaded, give it a timeout of 20 seconds
        WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, "//tbody/tr")))
        product_df = individual()
        master_df = pd.concat([master_df, product_df], ignore_index=True)
    except TimeoutException:
        # Handle the case where the table is not found within the specified timeout
        print(f"Timeout: Table not found for {product_link}")
    
    driver.close()
    driver.switch_to.window(main_window)

# Save the master_df to an Excel file
master_df.to_excel('output.xlsx', index=False)



Timeout: Table not found for https://world.openfoodfacts.org/product/8902080364053/mountain-dew-pepsico


In [13]:
master_df

,Product Name,Energy,Fat,Saturated fat,Carbohydrates,Sugars,Fiber,Proteins,Salt,Trans fat,...,Vitamin A,Paper or cardboard,Vitamin E,Vitamin C (ascorbic acid),Chloride,Zinc,Copper,Selenium,Folates (total folates),Cocoa (minimum)
0,Thums Up (Coca-Cola),176 kj\n(42 kcal),0 g,0 g,10.6 g,10.4 g,0 g,0 g,0.002 g,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Red bull energy drink - 250 ml,194 kj\n(46 kcal),0 g,0 g,11 g,11 g,0 g,0 g,0.1 g,0 g,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Sprite - coca cola - 750 ml,167 kj\n(40 kcal),0 g,0 g,10 g,9.8 g,0 g,0 g,0.014 g,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Heinz Tomato ketchup - 400ml,435 kj\n(102 kcal),0.1 g,< 0.1 g,23.2 g,22.8 g,?,1.2 g,1.8 g,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Thumbs Up - Coca Cola Company - 750ml,151 kj\n(36 kcal),0 g,0 g,9 g,9 g,?,0 g,0 g,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,Noix de cajou nature - Boni - 250 g,"2,431 kj\n(581 kcal)",44 g,7.8 g,27 g,5.9 g,3.3 g,18 g,0.03 g,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
95,Tamari soya sauce - Lima - 1 l,402 kj\n(96 kcal),0 g,0 g,5.3 g,1.7 g,0.9 g,12 g,10.3 g,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96,Maggi Masala Noodles - Nestle - 140g,"1,787 kj\n(427 kcal)",15.7 g,6.8 g,63.5 g,1.2 g,3.6 g,8 g,2.931 g,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97,kurkure - 25g,NaN,?,?,?,?,?,?,?,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [125]:
master_df.to_excel('output.xlsx', index=False)